In [53]:
%load_ext autoreload
%autoreload 2
import manenv
from manenv.utils.product_utils import *
from manenv.core.world import World
from manenv.core.component import *
from manenv.render_wrapper import RenderWrapper
from manenv.core.product import *
from manenv.core.effector import *

import manenv.effectors as eff
import manenv.components as cmp
import copy


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
world = World((5, 5))
s = np.zeros((5, 5), dtype=np.int8)
s[1][2] = 2
s[1][1] = 2
s[2][1] = 1
s[3][1] = 1
product = Product(s)
world.place_component(make_vector(1, 2), cmp.Conveyor(
    np.array([
    [0, -1, 0],
    [1, 0, 1],
    [0, 0, 0]
])))
world.place_component(make_vector(1, 1), cmp.Conveyor(
    np.array([
    [0, 0, 0],
    [1, 0, -1],
    [0, 0, 0]
])))
world.place_component(make_vector(2, 0), cmp.Conveyor(
    np.array([
    [0, -1, 0],
    [0, 0, 1],
    [0, 0, 0] 
])))
world.place_component(make_vector(1, 3), cmp.Conveyor(
    np.array([
    [0, 0, 0],
    [-1, 0, 1],
    [0, 0, 0]
])))
world.place_component(make_vector(2, 4), cmp.Conveyor(
    np.array([
    [0, -1, 0],
    [1, 0, 0],
    [0, 0, 0] 
])))
world.place_component(make_vector(2, 2), cmp.Conveyor(
    np.array([
    [0, 0, 0],
    [1, 0, -1],
    [0, 0, 0]
])))

world.place_component(make_vector(2, 1), cmp.Outport())

grabber = eff.Grabber(make_vector(0, 2))
buddy = eff.Grabber(make_vector(1, 3))
welder = eff.Welder(make_vector(1, 2))
transformer = eff.Transformer(1, 2, make_vector(2,  4))
discard = eff.Acceptor(make_vector(1, 7))

grabber_2 = eff.Grabber(make_vector(0, 2))
buddy_2 = eff.Grabber(make_vector(1, 3))
welder_2 = eff.Welder(make_vector(1, 2))
transformer_2 = eff.Transformer(1, 2, make_vector(2,  4))
discard_2 = eff.Acceptor(make_vector(1, 7))

world.place_component(make_vector(0, 2), cmp.Spawner(product))
assembler : cmp.Assembler = cmp.Assembler(make_vector(10, 10), [grabber, buddy, welder, discard])
assembler_2 : cmp.Assembler = cmp.Assembler(make_vector(10, 10), [grabber_2, buddy_2, welder_2, discard_2])
world.place_component(make_vector(1, 0), assembler)
# world.place_component(make_vector(1, 4), assembler_2)
wrapper = RenderWrapper(world)

In [55]:
wrapper.reset()
assembler.place_in_inventory(product.copy())
assembler.place_in_inventory(product.copy())
world.update()

grabber.set_action(eff.GrabberActions.GRAB_INVENTORY.value)
grabber.set_action(eff.GrabberActions.GRAB_INVENTORY.value)
buddy.set_action(eff.GrabberActions.GRAB_INVENTORY.value)
world.update()

grabber.set_action(eff.GrabberActions.RELEASE.value)
buddy.set_action(eff.GrabberActions.RELEASE.value)
world.update()

welder.set_action(eff.WelderActions.WELD_EAST.value)
world.update()
world.update()

grabber.set_action(eff.GrabberActions.GRAB.value)
world.update()

grabber.set_action(eff.GrabberActions.MOVE_RIGHT.value)
world.update()

grabber.set_action(eff.GrabberActions.RELEASE.value)
world.update()

# transformer.set_action(eff.TransformerActions.TRANSFORM.value)
# world.update()
# world.update()

# grabber.set_action(eff.GrabberActions.MOVE_RIGHT.value)
# world.update()
# world.update()

# grabber.set_action(eff.GrabberActions.MOVE_RIGHT.value)
# world.update()

# grabber.set_action(eff.GrabberActions.MOVE_RIGHT.value)
# world.update()


In [56]:
# wrapper.render()

# Gym Environment Test

In [57]:
from manenv.gym_wrapper import MARLFactoryEnvironment

In [58]:
# world.reset()
environment = MARLFactoryEnvironment(world)
renderer = RenderWrapper(environment._world)

In [59]:
from pettingzoo.test import parallel_api_test

In [60]:
def test():
    parallel_api_test(environment, num_cycles=1_000_000)

In [61]:
# test()

In [62]:
# import cProfile
# cProfile.run('test()', sort = 'time')

In [63]:
from manenv.solution.train_agents import train_loop
from manenv.solution.test_agents import test_agents

# Training Run

In [64]:
train_loop(environment, games=10000, seed=0)

Starting training on factory.
Using cuda device
-------------------------------
| time/              |        |
|    fps             | 7769   |
|    iterations      | 1      |
|    time_elapsed    | 42     |
|    total_timesteps | 327680 |
-------------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 3269        |
|    iterations           | 2           |
|    time_elapsed         | 200         |
|    total_timesteps      | 655360      |
| train/                  |             |
|    approx_kl            | 0.008419339 |
|    clip_fraction        | 0.0684      |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.3        |
|    explained_variance   | -0.00518    |
|    learning_rate        | 0.0003      |
|    loss                 | 16          |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00348    |
|    value_loss           | 79.4        |
--------------

KeyboardInterrupt: 

# Testing Run

In [14]:
test_agents(environment, games=1, seed=0)


Starting evaluation on factory (num_games=1)
FactoryMetrics(throughput={'12': 0}, utilization={'12': 0.0, '6': 0, '7': 0, '8': 0, '10': 0}, inventory={'12': 2.0}, cycle_time={'12': 0}, lead_time=0, customer_service=0, quality=0)
FactoryMetrics(throughput={'12': 0}, utilization={'12': 0.0, '6': 0, '7': 0, '8': 0, '10': 0}, inventory={'12': 2.0}, cycle_time={'12': 0}, lead_time=0, customer_service=0, quality=0)
FactoryMetrics(throughput={'12': 0}, utilization={'12': 0.25, '6': 1, '7': 0, '8': 0, '10': 0}, inventory={'12': 2.0}, cycle_time={'12': 0}, lead_time=0, customer_service=0, quality=0)
FactoryMetrics(throughput={'12': 0}, utilization={'12': 0.25, '6': 0, '7': 1, '8': 0, '10': 0}, inventory={'12': 2.0}, cycle_time={'12': 0}, lead_time=0, customer_service=0, quality=0)
FactoryMetrics(throughput={'12': 0}, utilization={'12': 0.25, '6': 0, '7': 1, '8': 0, '10': 0}, inventory={'12': 2.0}, cycle_time={'12': 0}, lead_time=0, customer_service=0, quality=0)
FactoryMetrics(throughput={'12'

3080.231730760776